## Data Preprocessing

In [16]:
import os
import pandas as pd
import numpy as np

In [2]:
path = '/nfs/turbo/intmed-bnallamo-turbo/wsliu/Data/UC_colonoscopy/'

Read the label file and build a dictionary: 

In [4]:
labels = pd.read_excel(path+'UC_Images_Labels/UC_deeplearning_imageLabels.xlsx')

In [9]:
labels.head()

,ID,basefile,pathname,file_seq,SubjectID,Time_sec,Video_time,Mayo_0,Mayo_1,Mayo_2,...,Findings_Waterjet,Findings_Diverticula,Location_Ileum,Location_Cecum,Location_Retroflexion,Comments,Poor_Image_UNCLASSIFIED,Quality_Good,Findings_Pills_retained,Location_appendicealOriface
0,119845,frame-000000.jpg,E:\Deep Learning - Endoscopy\001\frame-000000.jpg,1.0,001,1.0,00:00:01,False,False,False,...,False,False,False,False,False,NaN,True,False,False,False
1,119846,frame-000001.jpg,E:\Deep Learning - Endoscopy\001\frame-000001.jpg,2.0,001,2.0,00:00:02,False,False,False,...,False,False,False,False,False,NaN,True,False,False,False
2,119847,frame-000002.jpg,E:\Deep Learning - Endoscopy\001\frame-000002.jpg,3.0,001,3.0,00:00:03,False,False,False,...,False,False,False,False,False,NaN,True,False,False,False
3,119848,frame-000003.jpg,E:\Deep Learning - Endoscopy\001\frame-000003.jpg,4.0,001,4.0,00:00:04,False,False,False,...,False,False,False,False,False,NaN,True,False,False,False
4,119849,frame-000004.jpg,E:\Deep Learning - Endoscopy\001\frame-000004.jpg,5.0,001,5.0,00:00:05,False,False,False,...,False,False,False,False,False,NaN,True,False,False,False


In [7]:
labels.columns

Index(['ID', 'basefile', 'pathname', 'file_seq', 'SubjectID', 'Time_sec',
       'Video_time', 'Mayo_0', 'Mayo_1', 'Mayo_2', 'Mayo_3',
       'Poor_Image_underwater', 'Poor_Image_focus', 'Poor_Image_Stool',
       'Poor_Image_Bubbles', 'Post_Biopsy_Bleeding', 'Findings_BiopsyForceps',
       'Findings_Snare', 'Finding_Polyps', 'Finding_Pseudopolyps',
       'Findings_Waterjet', 'Findings_Diverticula', 'Location_Ileum',
       'Location_Cecum', 'Location_Retroflexion', 'Comments',
       'Poor_Image_UNCLASSIFIED', 'Quality_Good', 'Findings_Pills_retained',
       'Location_appendicealOriface'],
      dtype='object')

In [8]:
labels.Quality_Good.value_counts()

False    19102
True      7454
Name: Quality_Good, dtype: int64

In [14]:
labels_good = labels[labels.Quality_Good == True]

In [19]:
n_good = len(labels_good)
filepath = labels_good.pathname

In [22]:
filepath.values[0]

'E:\\Deep Learning - Endoscopy\\001\\frame-000243.jpg'

In [51]:
labels[(labels.Mayo_0 == True) & (labels.Quality_Good == False)].pathname.values

array(['E:\\Deep Learning - Endoscopy\\007\\frame-000023.jpg',
       'E:\\Deep Learning - Endoscopy\\007\\frame-000034.jpg',
       'E:\\Deep Learning - Endoscopy\\007\\frame-000109.jpg',
       'E:\\Deep Learning - Endoscopy\\007\\frame-000110.jpg',
       'E:\\Deep Learning - Endoscopy\\007\\frame-000127.jpg',
       'E:\\Deep Learning - Endoscopy\\007\\frame-000675.jpg',
       'E:\\Deep Learning - Endoscopy\\007\\frame-000732.jpg',
       'E:\\Deep Learning - Endoscopy\\007\\frame-000739.jpg',
       'E:\\Deep Learning - Endoscopy\\007\\frame-000799.jpg',
       'E:\\Deep Learning - Endoscopy\\007\\frame-000858.jpg',
       'E:\\Deep Learning - Endoscopy\\007\\frame-000967.jpg',
       'E:\\Deep Learning - Endoscopy\\007\\frame-000973.jpg',
       'E:\\Deep Learning - Endoscopy\\007\\frame-000975.jpg',
       'E:\\Deep Learning - Endoscopy\\007\\frame-001137.jpg',
       'E:\\Deep Learning - Endoscopy\\007\\frame-001479.jpg',
       'E:\\Deep Learning - Endoscopy\\007\\frame-00197

In [24]:
mayo = np.empty(n_good, dtype='int')
mayo[labels_good.Mayo_0 == True] = 0
mayo[labels_good.Mayo_1 == True] = 1
mayo[labels_good.Mayo_2 == True] = 2
mayo[labels_good.Mayo_3 == True] = 3

In [33]:
labels_nomayo = labels_good[(labels_good.Mayo_0 == False) & (labels_good.Mayo_1 == False) & (labels_good.Mayo_2 == False) & (labels_good.Mayo_3 == False)]

In [42]:
np.array(labels_nomayo.pathname)

array(['E:\\Deep Learning - Endoscopy\\007\\frame-000510.jpg',
       'E:\\Deep Learning - Endoscopy\\007\\frame-001059.jpg',
       'E:\\Deep Learning - Endoscopy\\007\\frame-002176.jpg',
       'E:\\Deep Learning - Endoscopy\\007\\frame-002538.jpg',
       'E:\\Deep Learning - Endoscopy\\045\\frame-001189.jpg',
       'E:\\Deep Learning - Endoscopy\\059\\frame-000488.jpg',
       'E:\\Deep Learning - Endoscopy\\059\\frame-001006.jpg',
       'E:\\Deep Learning - Endoscopy\\061\\frame-000445.jpg',
       'E:\\Deep Learning - Endoscopy\\061\\frame-000446.jpg',
       'E:\\Deep Learning - Endoscopy\\061\\frame-000447.jpg',
       'E:\\Deep Learning - Endoscopy\\061\\frame-000448.jpg',
       'E:\\Deep Learning - Endoscopy\\061\\frame-000449.jpg',
       'E:\\Deep Learning - Endoscopy\\061\\frame-000450.jpg',
       'E:\\Deep Learning - Endoscopy\\Lycera_001\\frame-000848.jpg',
       'E:\\Deep Learning - Endoscopy\\Lycera_001\\frame-001403.jpg',
       'E:\\Deep Learning - Endoscopy\\Ly

Move the files into corrsponding folders: 

In [35]:
dirName, subdirList, fileList = next(os.walk(path+'UC_Images_Labels'))

In [39]:
subdirList

['045',
 'Lycera_005-Deep Learning 1 FPS',
 '059',
 '060',
 '002',
 'Lycera_003-Deep Learning 1 FPS',
 '023',
 '007',
 '001',
 'Lycera_006-Deep Learning 1 FPS',
 'Lycera_004-Deep Learning 1 FPS',
 'Lycera_002-Deep Learning 1 FPS',
 '052',
 '008',
 '037',
 '046',
 '061',
 '036',
 '057',
 'Lycera_001-Deep Learning 1 FPS',
 '010']

In [40]:
for f in fileList:
    category = label_dict[f[:-5]]
    os.rename(path+'raw_data/'+f, path+'categorized/'+str(category)+'/'+f) # this is equivalent to $mv source destine

In [38]:
label_dict['20144_right']

4

In [39]:
labels.shape

(35126, 2)